<a href="https://colab.research.google.com/github/himanshoosaxena/himanshoosaxena/blob/main/Langchain_chains_agent_15thov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain setup

In [ ]:
!pip install openai langchain langchain-community

In [3]:
#secret Key

import os
from google.colab import userdata

# Retrieve API keys from Colab's secure storage

openai_api_key = userdata.get("OPENAI_API_KEY")

# Set them as environment variables

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

## Chains

Restaraunt Business Generator

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.9)

prompt_template_name= PromptTemplate(
    input_variables = ['cuisine'],
    template = "i want to open a restaraunt for  {cuisine} food. Suggest a fancy name for this"
                                         )
name_chain=LLMChain(llm=llm,prompt=prompt_template_name,output_key="restaraunt_name")

llm = OpenAI(temperature = 0.9)
prompt_template_items = PromptTemplate(input_variables=['restaraunt_name'],
                                      template="Suggest me some menu items for {restaraunt_name} which is {cuisine} cuisine. Only mention items names ,break down according to category like drinks soup starters main course"
                                       )
food_items_chain=LLMChain(llm=llm,prompt=prompt_template_items,output_key="menu_items")

from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables =['cuisine',],
    output_variables = ['restaraunt_name','menu_items']
)

In [5]:
chain({"cuisine": "Thai"})

/tmp/ipython-input-1763898740.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"cuisine": "Thai"})


{'cuisine': 'Thai',
 'restaraunt_name': '\n\n"Thai Temptations: A Modern Twist on Traditional Flavors"',
 'menu_items': ' and dessert\n\nDrinks:\n1. Thai Iced Tea\n2. Lemongrass Mojito\n3. Thai Ginger Beer\n4. Thai Basil Margarita\n5. Watermelon and Thai Basil Cooler\n6. Spicy Mango Lassi\n\nSoup:\n1. Tom Yum Soup\n2. Coconut Chicken Soup\n3. Spicy Seafood Soup\n4. Vegetable Tom Kha Soup\n5. Thai Noodle Soup\n6. Tom Klong Soup (spicy and sour broth with seafood and herbs)\n\nStarters:\n1. Thai Spring Rolls\n2. Spicy Beef Salad\n3. Chicken Satay with Peanut Sauce\n4. Thai Fish Cakes\n5. Grilled Shrimp Skewers\n6. Tofu Lettuce Wraps\n\nMain Course:\n1. Pad Thai\n2. Green Curry Chicken\n3. Massaman Beef Curry\n4. Pineapple Fried Rice\n5. Panang Tofu Curry\n6. Grilled Pork with Spicy Dipping Sauce\n7. Basil Chicken Stir-Fry\n8. Red Curry Duck\n9. Crispy Whole Fish with Chili Tamarind Sauce\n10. Spicy Eggplant and Tofu Stir-Fry\n\nDessert:\n'}

In [6]:
chain({"cuisine": "Italian"})

{'cuisine': 'Italian',
 'restaraunt_name': '\n\n"Pomodoro Elegante" ',
 'menu_items': '\n\nDrinks:\n1. Aperol Spritz\n2. Limoncello Martini \n3. Bellini \n4. White wine \n5. Italian soda \n\nSoup:\n1. Minestrone soup \n2. Tomato and basil soup \n3. Zuppa Toscana \n4. Stracciatella soup \n5. Mushroom risotto soup \n\nStarters: \n1. Bruschetta \n2. Caprese salad \n3. Prosciutto and melon \n4. Arancini \n5. Grilled calamari with lemon and herbs \n\nMain Course: \n1. Spaghetti carbonara \n2. Seafood risotto \n3. Chicken marsala \n4. Eggplant parmesan \n5. Gnocchi with gorgonzola and walnut cream sauce'}

In [ ]:
pip install gradio

In [8]:
import gradio as gr
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# DDS Logo URL
DDS_LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/airesidency/main/dds_logo.jpg"

# Define LLM chains as provided
llm = OpenAI(temperature=0.9)
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="i want to open a restaraunt for  {cuisine} food. Suggest a fancy name for this"
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaraunt_name")

llm = OpenAI(temperature=0.9)
prompt_template_items = PromptTemplate(
    input_variables=['restaraunt_name'],
    template="Suggest me some menu items for {restaraunt_name} which is {cuisine} cuisine. Only mention items names ,break down according to category like drinks soup starters main course"
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaraunt_name', 'menu_items']
)

# Function to run the chain and return outputs
def generate_restaurant(cuisine):
    outputs = chain({'cuisine': cuisine})
    return outputs['restaraunt_name'], outputs['menu_items']

# Build Gradio interface
with gr.Blocks(title="Restaurant Business Generator", theme="default") as demo:
    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            gr.Image(DDS_LOGO_URL, label="Decoding Data Science", elem_id="dds-logo")
            gr.Markdown("**Restaurant Business Generator**\nChoose a cuisine type and generate a restaurant name and menu instantly.")
            cuisine_input = gr.Textbox(label="Cuisine Type", placeholder="e.g. Italian, Japanese, Mexican")
            generate_btn = gr.Button("Generate")
        with gr.Column(scale=2):
            name_output = gr.Textbox(label="Suggested Restaurant Name", interactive=False)
            menu_output = gr.Textbox(label="Suggested Menu Items", lines=10, interactive=False)

    generate_btn.click(fn=generate_restaurant, inputs=cuisine_input, outputs=[name_output, menu_output])

if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c4d158e8f639cbc8d5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:


import gradio as gr
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# DDS Logo URL
DDS_LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/airesidency/main/dds_logo.jpg"

# Define LLM chains as provided
llm = OpenAI(temperature=0.9)
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="i want to open a restaraunt for  {cuisine} food. Suggest a fancy name for this"
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaraunt_name")

llm = OpenAI(temperature=0.9)
prompt_template_items = PromptTemplate(
    input_variables=['restaraunt_name', 'cuisine'],
    template="Suggest me some menu items for {restaraunt_name} which is {cuisine} cuisine. Only mention items names, break down according to category like drinks, soup, starters, main course"
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaraunt_name', 'menu_items']
)

# Popular cuisines dropdown options
TOP_CUISINES = [
    "Italian", "Chinese", "Japanese", "Mexican", "Indian",
    "French", "Thai", "Mediterranean", "American", "Spanish"
]

# Function to run the chain and return outputs
def generate_restaurant(cuisine):
    outputs = chain({'cuisine': cuisine})
    return outputs['restaraunt_name'], outputs['menu_items']

# Build Gradio interface
with gr.Blocks(title="Restaurant Business Generator", theme="default") as demo:
    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            gr.Image(DDS_LOGO_URL, label="Decoding Data Science", elem_id="dds-logo")
            gr.Markdown(
                "**Restaurant Business Generator**  \n"
                "Choose a cuisine type, or select from popular options, then generate a restaurant name and menu instantly."
            )
            cuisine_dropdown = gr.Dropdown(choices=TOP_CUISINES, label="Popular Cuisines", value=TOP_CUISINES[0])
            generate_btn = gr.Button("Generate")
        with gr.Column(scale=2):
            name_output = gr.Textbox(label="Suggested Restaurant Name", interactive=False)
            menu_output = gr.Textbox(label="Suggested Menu Items", lines=10, interactive=False)

    generate_btn.click(fn=generate_restaurant, inputs=cuisine_dropdown, outputs=[name_output, menu_output])

if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a630d919a7cfee43b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
import gradio as gr
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# DDS Logo URL
DDS_LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/airesidency/main/dds_logo.jpg"

# Define LLM chains as provided
llm = OpenAI(temperature=0.9)
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="i want to open a restaraunt for  {cuisine} food. Suggest a fancy name for this"
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaraunt_name")

llm = OpenAI(temperature=0.9)
prompt_template_items = PromptTemplate(
    input_variables=['restaraunt_name', 'cuisine'],
    template="Suggest me some menu items for {restaraunt_name} which is {cuisine} cuisine. Only mention items names, break down according to category like drinks, soup, starters, main course"
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaraunt_name', 'menu_items']
)

# Popular cuisines dropdown options
TOP_CUISINES = [
    "Italian", "Chinese", "Japanese", "Mexican", "Indian",
    "French", "Thai", "Mediterranean", "American", "Spanish"
]

# Function to run the chain and return outputs
def generate_restaurant(cuisine):
    outputs = chain({'cuisine': cuisine})
    return outputs['restaraunt_name'], outputs['menu_items']

# Wrapper to choose between dropdown and custom input
def on_generate(selected, custom):
    cuisine = custom.strip() if custom and custom.strip() else selected
    return generate_restaurant(cuisine)

# Build Gradio interface
with gr.Blocks(title="Restaurant Business Generator", theme="default") as demo:
    # Center-aligned app title
    gr.Markdown("<h1 style='text-align:center'>Restaurant Business Generator</h1>")
    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            # Logo resized to 300px width
            gr.Image(DDS_LOGO_URL, label="Decoding Data Science", elem_id="dds-logo", width=300)
            gr.Markdown(
                "Choose a cuisine from popular options or type your own, then generate a restaurant name and menu instantly."
            )
            cuisine_dropdown = gr.Dropdown(choices=TOP_CUISINES, label="Popular Cuisines", value=TOP_CUISINES[0])
            cuisine_textbox = gr.Textbox(label="Or enter custom cuisine", placeholder="e.g. Vietnamese, Peruvian")
            generate_btn = gr.Button("Generate")
        with gr.Column(scale=2):
            name_output = gr.Textbox(label="Suggested Restaurant Name", interactive=False)
            menu_output = gr.Textbox(label="Suggested Menu Items", lines=10, interactive=False)

    generate_btn.click(fn=on_generate, inputs=[cuisine_dropdown, cuisine_textbox], outputs=[name_output, menu_output])

if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6fd2cbce1b58b7435a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Agents

In [12]:
import os
from google.colab import userdata

SERPAPI_API_KEY = userdata.get("SERP_API_KEY")

# Set them as environment variables

if SERPAPI_API_KEY:
    os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

In [ ]:
pip install google-search-results

#### serpapi and llm-math tool

In [ ]:
pip install wikipedia

In [29]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi","wikipedia", "llm-math"], llm=llm)

'''
Why did the phrasing change the behaviour?
“add 5 to GDP” nudges the model toward “I should first figure out what GDP is numerically, and then just do +5”.
“add 5 trillion to this GDP” made it think in terms of a symbolic quantity (“GDP_2024”) then add 5e12.
This is a classic tool-use brittleness issue: the agent prompt and tool descriptions aren’t strict enough about
“calculator only accepts pure numeric expressions”.
'''
# Strengthen Calculator description
for t in tools:
    if t.name == "Calculator":  # name for llm-math tool
        t.description = (
            "Use this ONLY for math on explicit numbers. "
            "Input must be a pure mathematical expression with numbers only "
            "(like '1.2e13 + 5e12'), no words or variable names. "
            "It CANNOT look up real-world values (GDP, population, etc.). "
            "If you need something like GDP, FIRST use a search tool "
            "(SerpAPI or Wikipedia) to find the numeric value, then call this."
        )

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2020 and add 5 trillion to this GDP?")



> Entering new AgentExecutor chain...
 I should first use a search tool to find the GDP of US in 2020.
Action: Search
Action Input: "GDP of US in 2020"
Observation: ['Current-dollar GDP decreased 2.3 percent, or $500.6 billion, in 2020 to a level of $20.93 trillion, compared with an increase of 4.0 percent, ...', "23.681 trillion US dollars, a 10.9% increase from 2020. U.S. GDP for 2020 was 21.354 trillion US dollars, a 0.86% decline from 2019. GDP at purchaser's ...", 'GDP Growth Rate in the United States averaged 3.20 percent from 1947 until 2025, reaching an all time high of 34.90 percent in the third quarter of 2020 and a ...', 'GDP (current US$) - United States. Country official statistics, National ... 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007 ...', 'The US economy contracted 3.5 percent on an annual basis in 2020, the largest contraction for any full year since the demobilization from ...', 'Gross domestic product (GDP) is the market va

'The GDP of US in 2020 was 21.354 trillion and adding 5 trillion to this GDP gives a total of 26.354 trillion.'

In [33]:
from langchain.agents import AgentType, initialize_agent, Tool, load_tools
from langchain.llms import OpenAI
from langchain.utilities import WikipediaAPIWrapper

llm = OpenAI(temperature=0)

wiki = WikipediaAPIWrapper(
    top_k_results=1,               # only top result
    doc_content_chars_max=1000,    # truncate long pages
)

wikiTool = Tool(
        name="wikipedia",
        func=wiki.run,
        description=(
            "Use this to get brief background info from Wikipedia. "
            "It returns at most ~1000 characters, not full articles."
        ),
    )

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
# tools = load_tools(["serpapi", wikiTool, "llm-math"], llm=llm)

# Load standard tools (serpapi + llm-math)
# Make sure SERPAPI_API_KEY is set in your environment if you use serpapi
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Add our custom wikipedia tool
tools.append(wikiTool)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What is per capital income of dubai?")



> Entering new AgentExecutor chain...
 I should use a search engine to find the answer.
Action: Search
Action Input: "per capita income of dubai"
Observation: ['GDP per capita (current US$) - United Arab Emirates. Country official statistics, National Statistical Organizations and/or Central Banks; National Accounts ...', '$51,290 (nominal; 2025 est.) ... $82,000 (PPP; 2025 est.) GDP per capita rank.', 'UAE GDP per capita for 2022 was $49,899, a 15.08% increase from 2021. · UAE GDP per capita for 2021 was $43,360, a 16.64% increase from 2020. · UAE GDP per capita ...', 'GDP per Capita in the United Arab Emirates averaged 64097.50 USD from 1974 until 2024, reaching an all time high of 117949.38 USD in 1980 and a record low of ...', '\u2068Dubai\u2069 has a population of \u20681.9 million\u2069 people (\u20682020\u2069) and an estimated GDP per capita of $\u206840 thousand\u2069. Out of the \u2068348\u2069 cities covered in \u2068Asia\u2069, \u2068Dubai\u2069 ...', 'The gross domestic 

'The per capita income of Dubai is $62,014.'

#### Wikipedia and llm-math tool

In [ ]:
pip install wikipedia

In [26]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his current age in days")
agent.run("How many balloons can we fit in a380?")





> Entering new AgentExecutor chain...
 I should use Wikipedia to find out when Elon Musk was born
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (born June 28, 1971) is a businessman and entrepreneur known for his leadership of Tesla, SpaceX, Twitter, and xAI. Musk has been the wealthiest person in the world since 2021; as of October 2025, Forbes estimates his net worth to be around $500 billion.
Born into a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada; his Canadian citizenship is congenital, his mother having been born there. He received bachelor's degrees in 1997 from the University of Pennsylvania in Philadelphia, United States, before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also beca

ValueError: LLMMathChain._evaluate("
2021-06-28
") raised error: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<expr>, line 1). Please try again with a valid numerical expression